In [ ]:
from scipy.ndimage import gaussian_filter
import matplotlib.colors as mcolors
import copy
import os
import cv2
import imageio
import numpy as np
import pandas as pd
import pickle
# import scipy.stats as stats
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import *
mpl.rcParams['pdf.fonttype'] = 42
import seaborn as sns
# import scipy.stats
import tqdm
from scipy.stats.mstats import mquantiles
import torch
from numba import njit
import numpy as np
from scipy import ndimage

In [ ]:
pixels = pd.read_parquet("atlas.parquet")
pixels

## Define a function to do a radially exponentially decaying weighted neighbors interpolation with anatomical guide

In [ ]:
### adapted from Colas Droin

from numba import njit

@njit
def fill_array_interpolation(
    array_annotation,
    array_slices,
    divider_radius=5,
    annot_inside=-0.01,
    limit_value_inside=-2,
    structure_guided=True,
):
    """This function is used to fill the empty space (unassigned voxels) between the slices with
    interpolated values.

    Args:
        array_annotation (np.ndarray): Three-dimensional array of annotation coming from the Allen
            Brain Atlas.
        array_slices (np.ndarray): Three-dimensional array containing the lipid intensity values
            from the MALDI experiments (with many unassigned voxels).
        divider_radius (int, optional): Divides the radius of the region used for interpolation
            (the bigger, the lower the number of voxels used). Defaults to 5.
        annot_inside (float, optional): Value used to denotate the inside of the brain. Defaults
            to -0.01.
        limit_value_inside (float, optional): Alternative to annot_inside. Values above
            limit_value_inside are considered inside the brain. Defaults to -2.
        structure_guided (bool, optional): If True, the interpolation is done using the annotated
            structures. If False, the interpolation is done blindly.

    Returns:
        (np.ndarray): A three-dimensional array containing the interpolated lipid intensity values.
    """
    array_interpolated = np.copy(array_slices)

    for x in range(8, array_annotation.shape[0]): 
        for y in range(0, array_annotation.shape[1]):
            for z in range(0, array_annotation.shape[2]):
                # if we are in a unfilled region of the brain or just inside the brain
                condition_fulfilled = False
                if array_slices[x, y, z] >= 0:
                    condition_fulfilled = True
                elif limit_value_inside is not None and not condition_fulfilled:
                    if array_annotation[x, y, z] > limit_value_inside:
                        condition_fulfilled = True
                elif (
                    np.abs(array_slices[x, y, z] - annot_inside) < 10**-4
                ) and not condition_fulfilled:
                    condition_fulfilled = True
                if condition_fulfilled:
                    # check all datapoints in the same structure, and do a distance-weighted average
                    value_voxel = 0
                    sum_weights = 0
                    size_radius = int(array_annotation.shape[0] / divider_radius)
                    for xt in range(
                        max(0, x - size_radius), min(array_annotation.shape[0], x + size_radius + 1)
                    ):
                        for yt in range(
                            max(0, y - size_radius),
                            min(array_annotation.shape[1], y + size_radius + 1),
                        ):
                            for zt in range(
                                max(0, z - size_radius),
                                min(array_annotation.shape[2], z + size_radius + 1),
                            ):
                                # if we are inside of the sphere of radius size_radius
                                if (
                                    np.sqrt((x - xt) ** 2 + (y - yt) ** 2 + (z - zt) ** 2)
                                    <= size_radius
                                ):
                                    # the voxel has data
                                    if array_slices[xt, yt, zt] >= 0:
                                        # the structure is identical
                                        if (
                                            structure_guided
                                            and np.abs(
                                                array_annotation[x, y, z]
                                                - array_annotation[xt, yt, zt]
                                            )
                                            < 10**-4
                                        ) or not structure_guided:
                                            d = np.sqrt(
                                                (x - xt) ** 2 + (y - yt) ** 2 + (z - zt) ** 2
                                            )
                                            value_voxel += np.exp(-d) * array_slices[xt, yt, zt]
                                            sum_weights += np.exp(-d)
                    if sum_weights == 0:
                        pass
                    else:
                        value_voxel = value_voxel / sum_weights
                        array_interpolated[x, y, z] = value_voxel

    return array_interpolated

## Prepare useful functions and the Allen 3D anatomical reference

In [ ]:
def normalize_to_255(a):
    low_percentile_val = np.nanpercentile(a, 10)

    low_value_mask = a < low_percentile_val
    nan_mask = np.isnan(a)
    mask = np.logical_or(low_value_mask, nan_mask)

    a = np.where(mask, 0, a)

    a = ((a - a.min()) * (255 - 0) / (a.max() - a.min()))

    a[mask] = np.nan

    if not np.isnan(a).any():
        a = a.astype(np.uint8)

    return a

In [ ]:
from bg_atlasapi import BrainGlobeAtlas
atlas = BrainGlobeAtlas("allen_mouse_100um")

# reference image
reference_image = atlas.reference
print(reference_image.shape)

# annotation image
annotation_image = atlas.annotation
print(annotation_image.shape)

In [ ]:
import numpy as np

np.save("/data/luca/lipidatlas/euclid/euclid_msi/tests/reference_image100um.npy", reference_image)
np.save("/data/luca/lipidatlas/euclid/euclid_msi/tests/annotation_image100um.npy", annotation_image)

## Interpolate and smooth each lipid to yield visualizable 3D models

In [ ]:
from tqdm import tqdm
import os
from collections import defaultdict

lipids = np.array(pixels.columns)[:173]
pixels=pixels.dropna()

for xxx in tqdm(lipids):
    
    try:

        lipid_to_interpolate = pixels[["xccf", "yccf", "zccf", xxx]] 

        # logging the data seems to help rendering beautiful 3Ds
        lipid_to_interpolate.loc[:, xxx] = np.log(lipid_to_interpolate.loc[:, xxx])

        # prepare the data
        lipid_to_interpolate["xccf"] = lipid_to_interpolate["xccf"]*10
        lipid_to_interpolate["yccf"] = lipid_to_interpolate["yccf"]*10
        lipid_to_interpolate["zccf"] = lipid_to_interpolate["zccf"]*10

        tensor_shape = reference_image.shape

        tensor = np.full(tensor_shape, np.nan)

        intensity_col_name = lipid_to_interpolate.columns[3]

        intensity_values = defaultdict(list)

        for _, row in lipid_to_interpolate.iterrows():
            x, y, z = int(row["xccf"]) - 1, int(row["yccf"]) - 1, int(row["zccf"]) - 1
            intensity_values[(x, y, z)].append(row[intensity_col_name])

        for coords, values in intensity_values.items():
            x, y, z = coords
            if 0 <= x < tensor_shape[0] and 0 <= y < tensor_shape[1] and 0 <= z < tensor_shape[2]:
                tensor[x, y, z] = np.nanmean(values)

        not_nan_mask = np.logical_not(np.isnan(tensor))

        indices = np.where(np.any(not_nan_mask, axis=(1, 2)))

        normalized_tensor = normalize_to_255(tensor)

        w = 5

        non_nan_mask = ~np.isnan(normalized_tensor)
        normalized_tensor[non_nan_mask & (normalized_tensor < w)] = np.nan
        normalized_tensor[reference_image < 4] = 0

        # interpolate
        ahaha = fill_array_interpolation(array_annotation = annotation_image, array_slices = normalized_tensor)########, structure_guided=False)

        # clean up by convolution
        k = 10  # kernel size
        kernel = np.ones((k,k,k))
        array = np.where(np.isnan(ahaha), 0, ahaha)

        counts = np.where(np.isnan(ahaha), 0, 1)
        counts = ndimage.convolve(counts, kernel, mode='constant', cval=0.0)

        convolved = ndimage.convolve(array, kernel, mode='constant', cval=0.0)

        avg = np.where(counts > 0, convolved / counts, np.nan)

        filled_ahaha = np.where(np.isnan(ahaha), avg, ahaha)

        np.save(os.getcwd()+"/3d_interpolated_native/"+xxx+'interpolation_log.npy', filled_ahaha)
    
    except:
        continue